In [1]:
import numpy as np
import pandas as pd
import optuna
from sklearn.preprocessing import MinMaxScaler
import xgboost

import gower

from utils import encode_features, get_train_test_data, train_model, evaluate_model, generate_individual, epsilon_rounding, get_relevant_candidates

optuna.logging.set_verbosity(optuna.logging.WARNING)

/home/petron/AIMS/XAI/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

In [135]:
def load_data(data_filepath="../data/Loan_data_extracted.csv"):
    """
    Input: path to .csv data file

    TODO: specify in feature_info whether features are of type:
        fixed, meaning cannot change for the counterfactual
        unique, meaning can only take existing categorical values
        increase, meaning their value can only increase and not decrease
        range, meaning their new value can take a range of values

    Returns:
        dataframe and feature configuration dictionary
    """
    df = pd.read_csv(data_filepath)
    df = df.drop('Loan_ID', axis=1)
    df = df.dropna()

    feature_config = {
        "categorical": ["Gender", "Married", "Education", "Self_Employed", "Property_Area", "Loan_Status"],

        "feature_info": [
            ('Gender', 'fixed'),
            ('Married', 'unique'),
            ('Dependents', 'fixed'),
            ('Education', 'increase'),
            ('Self_Employed', 'unique'),
            ('ApplicantIncome', 'increase'),
            ('CoapplicantIncome', 'range'),
            ('LoanAmount', 'range'),
            ('Loan_Amount_Term', 'unique'),
            ('Credit_History', 'unique'),
            ('Property_Area', 'unique'),
        ],

        "categorical_features": ["Gender", "Married", "Education", "Self_Employed", "Property_Area"]
    }

    return df, feature_config

## Model

In [138]:
# Load the model from the saved file
model = xgboost.XGBClassifier()
model.load_model("../models/xgboost_model.json")

## TODO: Code for counterfactual search

In [95]:
def misfit(x_prime, y_target, model):
    """
    Optimisation criterion 1
    Calculate absolute difference between y_target and y_prime_prediction.

    This measures the desirability of the counterfactual
    """

    #TODO
    y_proba = model.predict_proba(x_prime.to_numpy().reshape(1, -1))[0][1]
    closeness = abs(y_proba - y_target)

    return closeness

In [96]:
def distance(X, x, x_prime, numerical, categorical):
    """
    Optimisation criterion 2
    Calculate distance between x_prime and x.

    This measures the closeness of the counterfactual
    """
    # Normalize data
    scaler = MinMaxScaler()
    scaler.fit(X[numerical])
    x_normalized = scaler.transform(x[numerical])
    x_prime_normalized = scaler.transform(x_prime[numerical])

    x_normalized = np.hstack((x_normalized, x[categorical].values.reshape(1, -1)))
    x_prime_normalized = np.hstack((x_prime_normalized, x_prime[categorical].values.reshape(1, -1)))

    # Compute distances
    #TODO
    gower_distance_matrix = gower.gower_matrix(np.vstack((
        x_prime_normalized,
        x_normalized
        )))
    distance = gower_distance_matrix[0, -1]

    return distance

In [97]:
def sparsity(x, x_prime):
    """
    Optimisation criterion 3
    Return number of unchanged features.

    This measures the sparsity of changes producing the counterfactual
    """
    #TODO

    num_no_diff = (x.values == x_prime.values).sum()

    return num_no_diff

In [98]:
def closest_real(X, x_prime, categorical, numerical):
    """
    Optimisation criterion 4
    Return the minimum distance between x_prime and any point in X.

    This measures the actionability of the counterfactual
    """
    scaler = MinMaxScaler()
    x_normalized = scaler.fit_transform(X[numerical])
    x_prime_normalized = scaler.transform(x_prime[numerical])

    x_normalized = x_normalized[X['Loan_Status']==1,::]
    X = X.loc[X['Loan_Status']==1, ::]

    x_normalized = np.hstack((x_normalized, X[categorical]))
    x_prime_normalized = np.hstack((x_prime_normalized, x_prime[categorical]))

    # Compute total distance
    #TODO
    # distance = (np.sqrt(np.sum(x_prime - X[anchor_idx])**2))     # Euclidean distance between x_prime and any point in x
    gower_dist_matrix =  gower.gower_matrix(np.vstack((            # Gower distance between x_prime and any point in x
        x_prime_normalized,
        x_normalized
    )))
    distance = min(gower_dist_matrix[0, 1:])

    return distance

In [107]:
def objective(trial, X, x, features, model, y_target, numerical, categorical):
    x_prime = x.copy()

    for feature in features:
        feature.sample(trial)
        x_prime[feature.name] = feature.value
    epsilon_rounding(x, x_prime, 1e-1)

    obj1 = misfit(x_prime, y_target, model)
    obj2 = distance(X, x, x_prime, numerical, categorical)
    obj3 = sparsity(x, x_prime)
    obj4 = closest_real(X, x_prime, categorical, numerical)

    return obj1, obj2, obj3, obj4

In [100]:
def get_counterfactuals(X, x, y_target, model,
                        numerical, categorical, features,
                        tol, optimization_steps, timeout):

    study = optuna.create_study(directions=['minimize', 'minimize', 'maximize', 'minimize'],
                                sampler=optuna.samplers.NSGAIISampler(seed=42))

    study.optimize(lambda trial: objective(trial, X, x, features, model,
                                y_target,
                                numerical,
                                categorical),
        n_trials=optimization_steps,
        timeout=timeout)

    candidates_df = get_relevant_candidates(study, x, model, y_target, tol)

    return candidates_df

## Provided datapoint and data

In [140]:
X_obs, feat_conf = load_data("../data/Loan_data_extracted.csv")
X_obs = encode_features(X_obs, feat_conf["categorical"])

--- 
Encoded categorical features as follows:
Gender :  {'Female': 0, 'Male': 1}
Married :  {'No': 0, 'Yes': 1}
Education :  {'Not Graduate': 0, 'Graduate': 1}
Self_Employed :  {'No': 0, 'Yes': 1}
Property_Area :  {'Rural': 0, 'Semiurban': 1, 'Urban': 2}
Loan_Status :  {'N': 0, 'Y': 1}
---


In [141]:
customer = np.array([0,1,0,0,0,2000,1500,1000,480,0,1])
x = pd.DataFrame([customer], columns=X_obs.columns[:-1].tolist())

In [142]:
# Check that our customer x did not get the loan

# TODO
y_pred = model.predict_proba(x)[:, 1].item()
print(f"Probability of user x getting loan: {y_pred:.4f}")

# and help her find out what she has to do in order to get the loan
# If you have implemented everything above correctly, the code below
# will find the counterfactuals

Probability of user x getting loan: 0.0030


## Search for counterfactuals

In [143]:
# Make a list of Feature objects containing information about how
# each feature is allowed to change when generating counterfactuals
change_features = generate_individual(X_obs, x, feat_conf["feature_info"])

In [144]:
# Set the desired new model prediction
y_CF = 0.7
print(f"Searching for counterfactuals with y_CF = {y_CF}...\n")
numerical_features = [x for x in X_obs.columns if x not in feat_conf["categorical"]]
CFS = get_counterfactuals(X_obs, x, y_CF, model,
                          numerical_features,
                          feat_conf["categorical_features"],
                          change_features,
                          tol=0.05,
                          optimization_steps=500,
                          timeout=None)

Searching for counterfactuals with y_CF = 0.7...


Predictions from 36 trials:
  Target: 0.7
  Tolerance: 0.05
  Min prediction: 0.007
  Max prediction: 0.993
  Mean prediction: 0.335
  Predictions within tolerance: 2


In [145]:
x

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,0,1,0,0,0,2000,1500,1000,480,0,1


In [146]:
CFS

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,0.0,1.0,0.0,0.0,0.0,2000.0,1563.97092,174.276386,180.0,1.0,1.0
1,0.0,1.0,0.0,0.0,0.0,2000.0,1563.97092,174.276386,360.0,1.0,1.0
